# 모듈 8: 멀티 에이전트(Multi-Agent) as Tool 구축하기

이 모듈에서는 하나의 거대 에이전트 대신, 여러 전문 에이전트가 협력하여 복잡한 문제를 해결하는 **멀티 에이전트 시스템**을 구축하는 방법을 배운다.

## 학습 목표

- **전문 에이전트**: 특정 도메인(날씨, 여행 등)에 특화된 에이전트를 만든다.
- **Agent-as-a-Tool**: 에이전트를 다른 에이전트의 도구로 변환하는 `AgentTool` 패턴을 이해한다.
- **위임(Delegation)**: 루트 에이전트가 하위 에이전트에게 작업을 맡기는 워크플로우를 구현한다.

## 라이브러리 불러오기

먼저, 이 노트북이 빌드된 특정 버전의 Google Agent Development Kit (ADK)를 설치한다. 버전을 고정하면 코드가 항상 예상대로 작동한다.

In [3]:
import os
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown

from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool
from google.genai.types import Content, Part
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session

## 인증: API 키 구성

다음으로 Google API 키를 안전하게 제공해야 한다. 이 코드는 키를 붙여넣을 수 있는 보안 입력 프롬프트를 생성한다. 그런 다음 키를 환경 변수로 설정하며, 이는 ADK가 요청을 인증하는 표준 방식이다.

In [4]:
load_dotenv()
MODEL = "gemini-2.5-flash"

## 3. 사용자 정의 도구 정의

여기서는 게스트 관리 에이전트를 위한 사용자 정의 스킬을 정의한다. 이것들은 게스트 데이터베이스 역할을 하는 딕셔너리와 상호 작용하는 간단한 파이썬 함수들이다. ADK는 함수 이름, 서명 및 독스트링을 사용하여 언제 어떻게 호출할지 이해한다.

In [5]:
# 게스트 목록 데이터베이스 역할을 하는 간단한 딕셔너리.
GUEST_DATABASE = {}

def add_guest(name: str, email: str) -> str:
  """게스트의 이름과 이메일을 이벤트 게스트 목록에 추가한다."""
  print(f"도구 실행됨: 이메일 '{email}'인 게스트 '{name}' 추가 중.")
  GUEST_DATABASE[email] = name
  return f"{name}을(를) 게스트 목록에 성공적으로 추가했다."

def get_guest_list() -> str:
  """이벤트에 등록된 모든 게스트의 현재 목록을 가져온다."""
  print("도구 실행됨: 게스트 목록 검색 중.")
  if not GUEST_DATABASE:
    return "게스트 목록이 현재 비어 있다."
  return json.dumps(GUEST_DATABASE)

## 전문가 에이전트 1 생성: 게스트 관리자

이것은 첫 번째 전문가 에이전트다. 유일한 업무는 이벤트의 게스트 목록을 관리하는 것이다. `add_guest`와 `get_guest_list` 함수를 도구로 제공하고 사용 방법에 대한 명확한 지침을 제공한다.

In [6]:
# 게스트 관리 전용 에이전트 정의
guest_management_agent = Agent(
    name="guest_management_agent",
    model=MODEL,
    description="이벤트의 게스트 목록을 관리하는 전문 에이전트다. 게스트를 추가하고 현재 목록을 검색할 수 있다.",
    instruction="""
    당신은 게스트 관리 어시스턴트다. 당신의 유일한 업무는 게스트를 목록에 추가하거나 요청 시 전체 목록을 가져오는 것이다.

    - add_guest 도구를 사용하여 새 사람을 목록에 추가한다.
    - get_guest_list 도구를 사용하여 현재 게스트 목록을 표시한다.

    이벤트 기획이나 정보 검색과 같은 다른 작업은 수행하지 않는다.
    도구를 사용하여 게스트 목록을 관리하는 것에만 엄격히 집중한다.
    """,
    tools=[add_guest, get_guest_list]
)

## 전문가 에이전트 2 생성: 이벤트 기획자

이것은 두 번째 전문가다. 내장된 Google 검색 도구를 사용하여 이벤트를 기획하는 데 전문성이 있다. 기획의 모든 측면을 처리하도록 지시받지만 게스트 관리에 대해서는 아무것도 모른다.

In [7]:
def create_event_planner_agent():
    """이벤트 기획자 에이전트 생성"""
    return Agent(
        name="event_planner_agent",
        model=MODEL,
        description="테마, 장소, 일정을 검색하여 이벤트를 기획하는 전문가 에이전트다. 사용자 피드백을 바탕으로 계획을 수정할 수 있다.",
        instruction="""
        당신은 전문 이벤트 기획자다. 당신의 유일한 목표는 이벤트를 위한 포괄적인 계획을 세우는 것이다.

        프로세스는 다음과 같다:
        1.  **테마 브레인스토밍:** 2-3가지 창의적인 테마를 제안한다.
        2.  **장소 찾기:** 검색 도구를 사용하여 3-5곳의 잠재적 장소를 찾는다.
        3.  **의제 작성:** 이벤트에 대한 높은 수준의 일정을 제안한다.
        4.  **명확하게 요약:** 최종 계획을 체계적인 형식으로 제시한다.

        게스트 목록이나 다른 작업은 처리하지 않는다. 기획에만 집중한다.

        항상 친절하게 대하고 사용자의 행동을 확인한다.
        """,
        tools=[google_search]
    )

event_planner_agent = create_event_planner_agent()
print(f"에이전트 '{event_planner_agent.name}'가 생성되어 준비되었다!")

에이전트 'event_planner_agent'가 생성되어 준비되었다!


## 에이전트 팀 구성

이것이 "도구로서의 에이전트(Agent-as-a-Tool)" 패턴의 핵심이다. `AgentTool`을 사용하여 두 명의 전문가 에이전트를 래핑하여 도구처럼 호출할 수 있게 만든다. 그런 다음 팀원들에게 작업을 위임하는 것이 유일한 임무인 "관리자" 에이전트, 즉 `root_orchestrator_agent`를 생성한다.

In [8]:
# 각 전문가 에이전트를 AgentTool로 래핑하여 메인 에이전트가 도구처럼 호출할 수 있게 한다.
event_planner_tool = AgentTool(agent=event_planner_agent)
guest_list_manager_tool = AgentTool(agent=guest_management_agent)

In [9]:
# 루트 오케스트레이터 정의: 사용자의 요청을 분석하여 적절한 도구(에이전트)로 라우팅한다.
root_orchestrator_agent = Agent(
    name="root_orchestrator_agent",
    model=MODEL,
    description="이벤트 기획 및 게스트 관리를 위해 전문 에이전트에게 작업을 위임하는 메인 오케스트레이터다.",
    instruction="""
    당신은 진행 중인 대화를 관리하는 루트 오케스트레이터다. 당신의 임무는 사용자 요청의 전체 맥락을 이해하고 올바른 전문 도구에 위임하는 것이다.

    - 사용자가 이벤트를 기획하거나, 장소를 찾거나, 일정을 만들려면 `event_planner_tool`을 사용한다.
    - 사용자가 게스트를 추가, 조회 또는 관리하려면 `guest_list_manager_tool`을 사용한다.

    사용자의 질문에 직접 대답하려고 하지 않는다. 항상 적절한 도구에 위임한다. 도구의 응답을 요약하지 않는다.

    """,
    tools=[event_planner_tool, guest_list_manager_tool]
)

## 실행 엔진 구축

이것은 쿼리를 실행하기 위한 헬퍼 함수다. Runner를 초기화하고 `run_async`로 이벤트를 스트리밍하며 최종 응답을 표시하는 핵심 ADK 로직을 처리한다. 이 함수를 사용하여 오케스트레이터와 상호 작용한다.

In [10]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Runner를 초기화하고 주어진 에이전트 및 세션에 대해 쿼리를 실행한다."""
    print(f"\n에이전트 쿼리 실행 중: '{agent.name}', 세션: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"오류 발생: {e}"

    print("\n" + "-"*50)
    print("최종 응답:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

## 세션 서비스 초기화 및 실행

마지막으로 `InMemorySessionService`를 설정하고 메인 실행 블록을 정의한다. 이 코드는 오케스트레이터를 위한 단일 세션을 생성한 다음, 올바른 전문가 에이전트에게 작업을 지능적으로 라우팅하는 방법을 보여주기 위해 일련의 쿼리를 보낸다.

In [11]:
# --- 세션 서비스 초기화 ---
# 이 하나의 서비스가 이 노트북의 모든 다른 세션을 관리한다.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [12]:
async def run_stateful_orchestrator():

  # 세션 생성
  session = await session_service.create_session(
        app_name=root_orchestrator_agent.name,
        user_id=user_id
  )

  # 1. 이벤트 기획 요청 (이벤트 기획자 에이전트로 라우팅됨)
  query1 = "뉴욕에서 AI/ML 모임을 기획해줘."
  print(f"사용자: {query1}\n")
  await run_agent_query(root_orchestrator_agent, query1, session, user_id)

  # 2. 게스트 추가 요청 (게스트 관리자 에이전트로 라우팅됨)
  query2 = "게스트 목록에 이메일 'tim.a@example.com'인 'Tim Apple'을 추가해줘."
  print(f"사용자: {query2}\n")
  await run_agent_query(root_orchestrator_agent, query2, session, user_id)

  # 3. 게스트 목록 확인 요청 (게스트 관리자 에이전트로 라우팅됨)
  query3 = "이제 게스트 목록을 보여줘."
  print(f"사용자: {query3}\n")
  await run_agent_query(root_orchestrator_agent, query3, session, user_id)

# 전체 시스템 실행
await run_stateful_orchestrator()

사용자: 뉴욕에서 AI/ML 모임을 기획해줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: '3998bddc-707c-4c56-b5e0-b3f15e0cd667'...



--------------------------------------------------
최종 응답:


뉴욕에서 AI/ML 모임을 기획하는 데 도움을 드릴 수 있어서 기쁩니다! 요청하신 내용에 따라 테마 브레인스토밍, 장소 물색, 의제 작성 및 최종 요약의 과정을 거쳐 포괄적인 계획을 세워보았습니다.

---

### **뉴욕 AI/ML 모임 기획 요약**

**1. 테마:**
*   **"AI 윤리와 미래 - 책임감 있는 AI 개발과 배포"**
*   **핵심 내용:** 빠르게 발전하는 AI 기술의 윤리적 함의에 초점을 맞춥니다. 데이터 프라이버시, 편향성, 투명성, 그리고 AI의 사회적 영향에 대한 논의를 심화하고, 책임감 있는 AI 개발을 위한 실질적인 접근 방안을 모색합니다. 다양한 산업 분야의 전문가들이 AI 윤리 가이드라인 및 모범 사례를 공유하는 자리가 될 것입니다.

**2. 잠재적 장소 (3-5곳):**
뉴욕 내에서 AI/ML 모임을 개최하기 적합한 잠재적 장소들을 탐색했습니다.

*   **Microsoft Technology Center (MTC) New York:** 최신 기술 시연 및 협업을 위한 공간으로, 기술 관련 이벤트 개최에 적합한 시설을 갖추고 있습니다. 미드타운 맨해튼에 위치해 접근성이 좋습니다. 특히, AI, 클라우드 솔루션 등 최첨단 기술 주제의 행사를 주최하며, 마이크로소프트 소호 지점에는 120명까지 수용 가능한 다목적 극장 '더 개러지'도 있습니다.
*   **Google NYC (Various Event Spaces):** 구글 뉴욕 캠퍼스에는 다양한 크기의 미팅룸과 이벤트 공간이 있어, 기술 커뮤니티 모임에 활용될 수 있습니다. 첼시에 위치하여 대중교통 이용이 편리합니다. (내부 공간 활용 여부는 직접 문의 필요)
*   **NYU Tandon School of Engineering:** 학술 기관의 강점을 활용하여 최첨단 연구 환경과 강연 시설을 제공할 수 있습니다. 브루클린 캠퍼스에는 100명 이상 수용 가능한 Pfizer Auditorium과 다목적 LC400룸, Maker EventSpace 등이 있습니다. 다만, 일부 공간은 NYU 제휴 관계자만 예약할 수 있습니다.
*   **The TimesCenter:** 뉴욕 타임스 빌딩 내에 위치한 전문 컨퍼런스 및 이벤트 공간으로, 현대적인 디자인과 최첨단 시청각 장비를 갖추고 있습니다. 378석 규모의 'The Stage' 강당과 5,000평방피트 규모의 유연한 'The Hall' 공간을 포함하며, 최대 500명까지 수용 가능합니다. 시간당 대여 방식으로 운영되며, 기술팀, 안내원, 보안 요원 등 인력이 포함됩니다.
*   **WorkBistro 및 기타 코워킹 스페이스:** Work Better NYC와 같은 코워킹 스페이스는 첼시에 11,000평방피트 규모의 이벤트 공간을 제공하며 최대 90명까지 수용할 수 있습니다. Workville은 미드타운 맨해튼에 옥상 공간을 갖춘 고급 이벤트 공간을 제공합니다. The Farm SoHo 등 다른 코워킹 스페이스들도 이벤트 공간을 제공합니다. 이들은 소규모에서 중규모 모임을 위한 유연한 공간을 제공하여 네트워킹을 촉진할 수 있습니다.

**3. 의제 (총 3시간 30분):**
"AI 윤리와 미래 - 책임감 있는 AI 개발과 배포" 테마에 맞춰 다음과 같은 일정을 제안합니다.

*   **18:00 - 18:30 (30분): 등록 및 네트워킹**
    *   참가자 등록, 환영 음료 및 다과, 자유 네트워킹.
*   **18:30 - 18:40 (10분): 오프닝 및 환영사**
    *   주최자 환영사, 모임 테마 및 목표 소개.
*   **18:40 - 19:20 (40분): 기조 강연 - "AI 윤리의 현재와 미래 과제"**
    *   AI 윤리 전문가의 인사이트 공유 (편향성, 투명성, 개인 정보 보호, 산업별 적용 사례 등).
*   **19:20 - 19:30 (10분): 휴식**
*   **19:30 - 20:10 (40분): 패널 토론 - "다양한 분야에서 책임감 있는 AI 구현"**
    *   기업, 학계, 법률 전문가 패널 토론 (AI 윤리 가이드라인, 규제 동향, 실질적 구현 전략), 질의응답.
*   **20:10 - 20:50 (40분): 심층 브레이크아웃 세션 (택 1)**
    *   **세션 A:** "데이터 편향성과 공정성 알고리즘 개발"
    *   **세션 B:** "AI 투명성과 설명 가능성(XAI) 기술 동향"
    *   **세션 C:** "AI 규제 프레임워크와 법적 책임"
*   **20:50 - 21:00 (10분): 마무리 및 클로징**
    *   세션 요약, 핵심 메시지 강조, 향후 모임 안내, 감사 인사.
*   **21:00 이후: 비공식 네트워킹**
    *   자유로운 추가 네트워킹 시간.

---

이 계획은 뉴욕에서 성공적인 AI/ML 모임을 개최하기 위한 탄탄한 기반이 될 것입니다. 이 계획에 대해 궁금하시거나 수정하고 싶은 부분이 있으시면 언제든지 말씀해주세요!

--------------------------------------------------

사용자: 게스트 목록에 이메일 'tim.a@example.com'인 'Tim Apple'을 추가해줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: '3998bddc-707c-4c56-b5e0-b3f15e0cd667'...
도구 실행됨: 이메일 'tim.a@example.com'인 게스트 'Tim Apple' 추가 중.

--------------------------------------------------
최종 응답:


Tim Apple 님을 게스트 목록에 추가했습니다.

--------------------------------------------------

사용자: 이제 게스트 목록을 보여줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: '3998bddc-707c-4c56-b5e0-b3f15e0cd667'...
도구 실행됨: 게스트 목록 검색 중.

--------------------------------------------------
최종 응답:


현재 게스트 목록은 다음과 같습니다: Tim Apple (tim.a@example.com)

--------------------------------------------------

